In [1]:
from git_request import search_top_starred_repositories, search_best_match_repositories
from openml_request import search_top_related_local_datasets_with_chatgpt, search_newest_datasets, search_top_related_local_datasets_with_cs
from gpt_request import get_response_from_chatgpt_with_context
from localgpt_request import load_text_generation_pipeline

import time
import re
import pandas as pd
import numpy as np 

/pfs/data5/home/kit/tm/hj7422/conda/envs/vicuna/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# these package
import requests
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import Chroma

we can also seperate these parckage and import them when necessary. Maybe add some explaination of the function when import

In [3]:
import sdsc_cataglog

## Test text generation model

In [5]:
#model_name = "mistralai/Mistral-7B-Instruct-v0.1" #一般
#model_name = "Open-Orca/Mistral-7B-OpenOrca" # 333s 结果还行
#model_name = "HuggingFaceH4/zephyr-7b-alpha" # 这个挺好，时间上也是很长 151
#model_name = "meta-llama/Llama-2-7b" # no config.json
#model_name = "mistralai/Mistral-7B-Instruct-v0.1" #等待时间过长
#model_name = "replit/replit-code-v1-3b" # ReplitLMTokenizer not found
#model_name = "liuhaotian/llava-v1.5-13b" # keyerror llava
#model_name = "stabilityai/stablelm-3b-4e1t" # str object is not callable
#model_name = "lmsys/vicuna-7b-v1.5" 不行
#model_name = "teknium/Mistral-Trismegistus-7B" 等待时间过长
#model_name = "gpt2"  效果不行
model_name = "meta-llama/Llama-2-7b-chat-hf" # 挺好，就是等得久152s
#TO test: liuhaotian/llava-v1.5-13b

In [6]:
t1 = time.time()
nlp = load_text_generation_pipeline()
t2 = time.time()
t2-t1

Loading checkpoint shards: 100%|██████████| 2/2 [00:39<00:00, 19.77s/it]
/pfs/data5/home/kit/tm/hj7422/conda/envs/vicuna/lib/python3.10/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


72.68264412879944

Here we will download and built a local gpt model for text generation. The model we selected is about 15GB, so it will take a while. 

introduce about the nlp and what kinds of model we will use and why here...

In [7]:
use_case_e = 'This task was associated with two difficulties: 1) the requirement of interpretability limits the choice of models and 2) the large number of attributes stored as text complicates the analysis and at the same time affects the performance of some models.There are many complex models whose decisions are difficult to explain, deep neural networks are one of the typical examples. Attributes stored using text are mostly contextual and can be handled well by semantic analysis, but it requires a large amount of data and time and is not interpretable. Instead, some simple methods can add or remove information.'
use_case = 'Diese Aufgabe war mit zwei Schwierigkeiten verbunden: 1) die Anforderung der Interpretierbarkeit schränkt die Auswahl der Modelle ein und 2) die große Anzahl als Text gespeicherter Merkmale erschwert die Analyse und beeinträchtigt gleichzeitig die Leistung einiger Modelle.Es gibt viele komplexe Modelle, deren Entscheidungen schwer zu erklären sind, tiefe neuronale Netze sind eines der typischen Beispiele dafür. Attribute, die mit Hilfe von Text gespeichert werden, sind meist kontextabhängig und können durch semantische Analyse gut behandelt werden, aber sie erfordert eine große Menge Daten und Zeit und ist nicht interpretierbar. Stattdessen können bei einigen einfachen Methoden Informationen hinzugefügt oder entfernt werden.'

Here is the use case i get from our website. Since the llm is not as powerful as the one online. Only english work here.

Maybe add some explaination of the use case here. What is the use case about and what kinds of challenges is he faccing?

In [8]:
# nlp(question = QA_input['question'], context = QA_input['context'])
# 
t1 = time.time()
rsl = nlp("I got a use case: " + use_case_e + " I can summarise the chanllenges mentioned in this use case with three keywords: ",
          num_return_sequences = 1,
          # temperature = 1.0,
          # num_beams = 5, # 
          # top_k =  50
         )
t2 = time.time()
t2-t1
#nlp(messages)

138.83806991577148

we will limit some function of the model, so that the model will not take too long to wait.

explain some parameter and their effect here...

The most popular size of llm used locally is 3b, 7b, 30b, 70b. Here, 3b means the model contain 3 billion parameters. The larger the model, the better its performance. The size of chatgpt we used online is larger than 70b. Therefore, it's performance is better than the one we use here. For this presentation, due to the limitation of applied resource, we use a llm with about 7 billion parameters. The size of the model is about 15GB. And one single request takes about 2 mins. This prcess can be accelerated through torchserver and parallelization. 

talk about the acceleration story....

The llm model here is text generation. text generation model always add text to the original data which make the extraction of the answer difficult. To simplify the task, we need to add a special sign at the end of the given sentence. This sign should not affect the meaning of the text and should not be contain in the test. We use '::' here. 

talk about the promt generation...

In [9]:
rsl[0]

{'generated_text': 'I got a use case: This task was associated with two difficulties: 1) the requirement of interpretability limits the choice of models and 2) the large number of attributes stored as text complicates the analysis and at the same time affects the performance of some models.There are many complex models whose decisions are difficult to explain, deep neural networks are one of the typical examples. Attributes stored using text are mostly contextual and can be handled well by semantic analysis, but it requires a large amount of data and time and is not interpretable. Instead, some simple methods can add or remove information. I can summarise the chanllenges mentioned in this use case with three keywords: 1) interpretability, 2) complexity, and 3) textual attributes.\n\n\n'}

In [10]:
pattern = r'\d+\.|\)\s+([^0-9|,]+)(?=\d+|\.|\s|,|$)'
keywords = re.findall(pattern, rsl[0]['generated_text'].strip().split('keywords:')[-1].strip())
keywords

['interpretability', 'complexity', 'textual attributes.']

Since the output of the llm model are always not uniform. In this example, it use ')' after the number, however, sometimes it use the '.'. This can enlarge the difficult to extract the keyword. Here we use the 're' package to do the job.

Add some explaination about the re pattern used here....

Go back to our main story line. Here, for each keyward, we want to .....

In [11]:
# import requests
# def search_best_match_repositories(keyword, num = 5):
#     url = "https://api.github.com/search/repositories"
#     headers = {
#         "Accept": "application/vnd.github.v3+json"
#     }
#     params = {
#         "q": keyword,
#         "sort": "best_match",  # 按星数排序
#         "order": "desc",  # 降序排列
#         "per_page": num    # 获取前5个结果
#     }
#     response = requests.get(url, headers=headers, params=params)
#     if response.status_code == 200:
#         data = response.json()
#         repositories = data["items"]
#         repo_urls = []
#         readme_urls = []
#         for repo in repositories:
#             repo_url = repo["html_url"]
#             repo_urls.append(repo_url)
#             readme_urls.append(repo_url + "/blob/master/README.md")
#             #stars_count = repo["stargazers_count"]

#         return repo_urls, readme_urls
#     else:
#         print("Error:", response.status_code)
#     return None, None

In [12]:
for keyword in keywords:
    git_urls, readme_urls = search_best_match_repositories(keyword+' python') # +' problem'
    if git_urls is not None:
        print(f"For the {keyword} preblem, we recommend the following tools:")
        for git_url, readme_url in zip(git_urls, readme_urls):
            print("Repository URL:", git_url)
            print("README URL:", readme_url)
        print('-'*50)

For the interpretability preblem, we recommend the following tools:
Repository URL: https://github.com/jphall663/awesome-machine-learning-interpretability
README URL: https://github.com/jphall663/awesome-machine-learning-interpretability/blob/master/README.md
Repository URL: https://github.com/MAIF/shapash
README URL: https://github.com/MAIF/shapash/blob/master/README.md
Repository URL: https://github.com/h2oai/mli-resources
README URL: https://github.com/h2oai/mli-resources/blob/master/README.md
Repository URL: https://github.com/PacktPublishing/Interpretable-Machine-Learning-with-Python
README URL: https://github.com/PacktPublishing/Interpretable-Machine-Learning-with-Python/blob/master/README.md
Repository URL: https://github.com/interpretml/interpret-community
README URL: https://github.com/interpretml/interpret-community/blob/master/README.md
--------------------------------------------------
For the complexity preblem, we recommend the following tools:
Repository URL: https://git

There are two different ways to get the advises. The first one is used the api provied by the corresponding website. The example above is using the git api to get the relavant tools. We can see that it works good for the interpretability keywords. The advised tools can help to make the code or data more interpretable. It is due to the core of the search stragety implemented by the git api tool. A normal git repository contains information include: 'readme', 'keywords', and 'about'. The api will looking for the given keyword in these files and send the repository back when it is hit. However, it will not help for the second keyword 'complexity' since the word itself miss the context information. According to a single keyword it is hard to understand what problem we are dealing with. In this case, we will preper the second way to get the recommendation, using the embedding.

In [4]:
#for keyword in keywords:
keyword = 'textual attributes'
dataset_names, dataset_ids, dataset_description, dataset_urls = search_top_related_local_datasets_with_cs([keyword])
print(f"For the {keyword} preblem, we recommend the following datasets:")
for ds_name, ds_des, ds_url in zip(dataset_names, dataset_description, dataset_urls):
    print("Dataset Name:", ds_name)
    print("Dataset description", ds_des)
    print("Dataset URL:", ds_url)
print('-'*50)

load INSTRUCTOR_Transformer
max_seq_length  512
For the textual attributes preblem, we recommend the following datasets:
Dataset Name: analcatdata_authorship
Dataset description The dataset analcatdata_authorship contains information related to the authorship of various texts. It includes a total of 841 instances, each representing a text document. There are five attributes included in this dataset, which are used to categorize the texts. These attributes include the age of the text, the origin of the text, the type of text, its length, and its author. The dataset is intended to be used for authorship attribution analysis or any other related research that involves text analysis, classification, or identification of authorship patterns.
Dataset URL: https://www.openml.org/d/970.0
Dataset Name: page-blocks
Dataset description The dataset "page-blocks" consists of scanned grayscale images of document pages. Each image is divided into equally sized blocks and labeled as either a text or n

In this example, we use the embedding to get the recommendation on dataset. The working process is as follow, 1. we download the descprition of all the dataset and generate word embedding according to the description. We save the dataset, id, descrption and embedding on the local disk. 2. we get the key word or problem description, use the llm to generate embedding of the given information. 3. We go through the local dataset and compare the embedding similarity. 4. We rank the similarity and get the top k as feedback. There are several advantage of using such process. 1. The keyword doesn't have to hided in the name or description. It's important since different word may have the same meansing e.g., explanability and interpretability. 2. We support to use the problem description as input, which may contain some important context information. 3. it doesn't rely on certain api. Taking openml api for example, it doesn't support blur search. 4. It can be achieved off-line. 
